## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '../..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://github.com/anhaidgroup/deepmatcher/blob/master/Datasets.md

In [6]:
import glob
import csv
from tqdm.auto import tqdm

from entity_embed.data_utils.utils import Enumerator

row_dict = {}
id_enumerator = Enumerator()
left_id_set = set()
right_id_set = set()
rows_total = 2554 + 22074
clusters_total = 1154

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/tableA.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'left-{int(row["id"])}']
            row['source'] = 'left'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/tableB.csv') as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[f'right-{int(row["id"])}']
            row['source'] = 'right'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [7]:
train_true_pair_set = set()
valid_true_pair_set = set()
test_true_pair_set = set()

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/train.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            train_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/valid.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            valid_true_pair_set.add((id_left, id_right))

with open(f'{home_dir}/Downloads/walmart_amazon_exp_data/exp_data/test.csv') as f:
    for row in csv.DictReader(f):
        if int(row['label']) == 1:
            id_left = id_enumerator[f'left-{int(row["ltable_id"])}']
            id_right = id_enumerator[f'right-{int(row["rtable_id"])}']
            test_true_pair_set.add((id_left, id_right))
        
display(('train_true_pair_set', len(train_true_pair_set)))
display(('valid_true_pair_set', len(valid_true_pair_set)))
display(('test_true_pair_set', len(test_true_pair_set)))

('train_true_pair_set', 576)

('valid_true_pair_set', 193)

('test_true_pair_set', 193)

## Preprocess

In [8]:
attr_list = ['title', 'category', 'brand', 'modelno', 'price']

In [9]:
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'title_semantic': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'category': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
        'use_mask': True,
    },
    'category_semantic': {
        'source_attr': 'category',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'brand': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'modelno': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [13]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

17:10:25 INFO:For attr=title, computing actual max_str_len
17:10:25 INFO:actual_max_str_len=23 must be pair to enable NN pooling. Updating to 24
17:10:25 INFO:For attr=title, using actual_max_str_len=24
17:10:26 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:10:29 INFO:For attr=category, computing actual max_str_len
17:10:29 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
17:10:29 INFO:For attr=category, using actual_max_str_len=16
17:10:29 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:10:32 INFO:For attr=brand, computing actual max_str_len
17:10:32 INFO:For attr=brand, using actual_max_str_len=48
17:10:32 INFO:For attr=modelno, computing actual max_str_len
17:10:32 INFO:For attr=modelno, using actual_max_str_len=48
17:10:32 INFO:For attr=price, computing actual max_str_len
17:10:32 INFO:For attr=price, using actual_max_str_len=10


{'title': NumericalizeInfo(attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=24, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'title_semantic': NumericalizeInfo(attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"',

In [14]:
from entity_embed import PairwiseDataModule

datamodule = PairwiseDataModule(
    row_dict=row_dict,
    row_numericalizer=row_numericalizer,
    batch_size=10,
    row_batch_size=16,
    train_true_pair_set=train_true_pair_set,
    valid_true_pair_set=valid_true_pair_set,
    test_true_pair_set=test_true_pair_set,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = 'f1-walmart-amazon'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

17:10:33 INFO:GPU available: True, used: True
17:10:33 INFO:TPU available: None, using: 0 TPU cores
17:10:33 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

17:10:35 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 22.6 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
13.7 M    Trainable params
8.9 M     Non-trainable params
22.6 M    Total params


17:10:35 WARNING:true_pair_set has 38 less elements than transitive_true_pair_set. This means there are transitive true pairs not included in true_pair_set.


1

In [18]:
model.blocker_net.get_signature_weights()

{'title': 0.28698351979255676,
 'title_semantic': 0.24623051285743713,
 'category': 0.05953279137611389,
 'category_semantic': 0.04035833477973938,
 'brand': 0.17525532841682434,
 'modelno': 0.14158497750759125,
 'price': 0.050054535269737244}

In [19]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.42464246424642466,
 'valid_f1_at_0.5': 0.7901234567901234,
 'valid_f1_at_0.7': 0.9343832020997374,
 'valid_f1_at_0.9': 0.5703703703703703,
 'valid_pair_entity_ratio_at_0.3': 1.8891820580474934,
 'valid_pair_entity_ratio_at_0.5': 0.7730870712401056,
 'valid_pair_entity_ratio_at_0.7': 0.49604221635883905,
 'valid_pair_entity_ratio_at_0.9': 0.20316622691292877,
 'valid_precision_at_0.3': 0.26955307262569833,
 'valid_precision_at_0.5': 0.6552901023890785,
 'valid_precision_at_0.7': 0.9468085106382979,
 'valid_precision_at_0.9': 1.0,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9948186528497409,
 'valid_recall_at_0.7': 0.9222797927461139,
 'valid_recall_at_0.9': 0.39896373056994816}

## Testing

In [20]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.48614609571788414,
  'test_f1_at_0.5': 0.8584269662921349,
  'test_f1_at_0.7': 0.8999999999999999,
  'test_f1_at_0.9': 0.5210727969348659,
  'test_pair_entity_ratio_at_0.3': 1.5651041666666667,
  'test_pair_entity_ratio_at_0.5': 0.65625,
  'test_pair_entity_ratio_at_0.7': 0.4869791666666667,
  'test_pair_entity_ratio_at_0.9': 0.17708333333333334,
  'test_precision_at_0.3': 0.3211314475873544,
  'test_precision_at_0.5': 0.7579365079365079,
  'test_precision_at_0.7': 0.9144385026737968,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 0.9896373056994818,
  'test_recall_at_0.7': 0.8860103626943006,
  'test_recall_at_0.9': 0.35233160621761656}]

## Testing manually 

In [21]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict, cluster_dict_to_filtered_row_dict

__, test_cluster_dict = id_pairs_to_cluster_mapping_and_dict(test_true_pair_set)
test_row_dict = cluster_dict_to_filtered_row_dict(row_dict, test_cluster_dict)
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [22]:
embedding_size = model.blocker_net.embedding_size

In [23]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [24]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 648 ms, sys: 10.7 ms, total: 659 ms
Wall time: 113 ms


In [37]:
%%time

sim_threshold = 0.7
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 345 ms, sys: 0 ns, total: 345 ms
Wall time: 37.4 ms


In [38]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

0.4869791666666667

In [39]:
from entity_embed.evaluation import precision_and_recall, f1_score

precision, recall = precision_and_recall(found_pair_set, test_true_pair_set)
precision, recall

(0.9144385026737968, 0.8860103626943006)

In [40]:
f1_score(precision, recall)

0.8999999999999999

In [41]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

16

In [42]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

22

In [31]:
cos_similarity = lambda a, b: np.dot(a, b)

In [32]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.68038434,
 {'id': 1613,
  'title': 'pinnacle studio hd v15 ultimate collection pc',
  'category': 'software',
  'brand': 'avid technology',
  'modelno': '',
  'price': '119 . 0',
  'source': 'left'},
 {'id': 19772,
  'title': 'pinnacle studio ultimate collection v . 15',
  'category': 'computers accessories',
  'brand': 'avid',
  'modelno': '82103002401',
  'price': '99 . 99',
  'source': 'right'})

(0.594212,
 {'id': 2315,
  'title': 'seiko slp - mrl multipurpose label',
  'category': 'stationery & office machinery',
  'brand': 'seiko',
  'modelno': 'slp - mrl',
  'price': '14 . 88',
  'source': 'left'},
 {'id': 12102,
  'title': 'smart label printer multipurpose labels',
  'category': 'audio video accessories',
  'brand': 'seiko',
  'modelno': '',
  'price': '13 . 82',
  'source': 'right'})

(0.58667594,
 {'id': 1810,
  'title': 'microsoft comfort optical mouse 3000 silver blue',
  'category': 'mice',
  'brand': 'microsoft',
  'modelno': 'd1t00011',
  'price': '19 . 95',
  'source': 'left'},
 {'id': 3131,
  'title': 'comfort opt mse3000 silver blu',
  'category': 'computers accessories',
  'brand': 'microsoft',
  'modelno': 'd1t - 00011',
  'price': '11 . 99',
  'source': 'right'})

(0.32196838,
 {'id': 1396,
  'title': 'dj - tech t545a 15 600w 2 - way active loudspeaker',
  'category': 'stereos / audio',
  'brand': 'dj tech',
  'modelno': 't545a',
  'price': '358 . 0',
  'source': 'left'},
 {'id': 4992,
  'title': 'djtech super powered 15 600 w dj spkrs',
  'category': 'home audio theater',
  'brand': 'dj - tech',
  'modelno': 't545a',
  'price': '396 . 74',
  'source': 'right'})

(0.6114865,
 {'id': 2118,
  'title': 'antec tricool 92mm double ball bearing case fan',
  'category': 'electronics - general',
  'brand': 'antec',
  'modelno': 'tricool 92mm dbb',
  'price': '8 . 25',
  'source': 'left'},
 {'id': 23175,
  'title': 'antec tricool 92mm dbb case fan with 3 - spd switch 3 4 - pin 3 - pin monitoring',
  'category': '',
  'brand': 'antec',
  'modelno': 'tricool 92mm dbb',
  'price': '8 . 25',
  'source': 'right'})

(0.568462,
 {'id': 1549,
  'title': 'comprehensive 1 4 standard phone plug to 3 - pin xlr jack set of 10',
  'category': 'tv accessories',
  'brand': 'comprehensive',
  'modelno': 'spp - xlrj',
  'price': '79 . 9',
  'source': 'left'},
 {'id': 21502,
  'title': 'comprehensive premium - audio adapter - mini - phone 3 . 5 mm m - 3 pin xlr f',
  'category': 'connectors adapters',
  'brand': 'comprehensive video',
  'modelno': '',
  'price': '',
  'source': 'right'})

(0.68681264,
 {'id': 1829,
  'title': 'viewsonic pocket camcorder projector',
  'category': 'camcorders',
  'brand': 'viewsonic',
  'modelno': '3dv5',
  'price': '129 . 98',
  'source': 'left'},
 {'id': 21907,
  'title': 'viewsonic 3dv5 pocket 3d hd camcorder with 5m ff lens',
  'category': 'camcorders',
  'brand': 'viewsonic',
  'modelno': '3dv5',
  'price': '99 . 0',
  'source': 'right'})

(0.5659901,
 {'id': 2140,
  'title': 'htc sensation 4g',
  'category': 'prepaid wireless phones',
  'brand': 'generic',
  'modelno': '610214626400',
  'price': '0 . 0',
  'source': 'left'},
 {'id': 9519,
  'title': 'htc sensation 4g android phone t - mobile',
  'category': 'phones with plans',
  'brand': 'htc',
  'modelno': 'sensation',
  'price': '',
  'source': 'right'})

(0.6252664,
 {'id': 498,
  'title': 't - mobile samsung sgh - t249 prepaid slider camera phone with bluetooth',
  'category': 'prepaid wireless phones',
  'brand': 't - mobile',
  'modelno': 't249',
  'price': '49 . 84',
  'source': 'left'},
 {'id': 8131,
  'title': 'samsung t249 prepaid phone blue t - mobile',
  'category': 'no - contract phones',
  'brand': 'samsung',
  'modelno': 't239',
  'price': '59 . 99',
  'source': 'right'})

(0.6113563,
 {'id': 33,
  'title': 'sunpak 72 - in - one card reader with sim slot',
  'category': 'photography - general',
  'brand': 'sunpak',
  'modelno': '72 - in - 1 - cr',
  'price': '0 . 0',
  'source': 'left'},
 {'id': 14237,
  'title': 'sunpak 72 - in - 1 high speed card reader - universal usb 2 . 0 card reader including sim',
  'category': 'memory card readers',
  'brand': 'sunpak',
  'modelno': '',
  'price': '19 . 95',
  'source': 'right'})